## Import necessary libraries

In [11]:
import pandas as pd
import numpy as np
import sys
import tensorflow as tf
from sklearn.metrics import f1_score

sys.path.append('../../src')
from rnn.rnn_from_scratch import SimpleRNNModel
from utils.data_preprocessing import TextPreprocessor
from rnn.model_training import create_keras_rnn

import warnings
warnings.filterwarnings("ignore")


In [12]:
train = pd.read_csv('../../datasets/train.csv', index_col='id')
valid = pd.read_csv('../../datasets/valid.csv', index_col='id')
test = pd.read_csv('../../datasets/test.csv', index_col='id')
train.head()

,text,label
id,,
219,Nikmati cicilan 0% hingga 12 bulan untuk pemes...,neutral
209,Kue-kue yang disajikan bikin saya bernostalgia...,positive
436,Ibu pernah bekerja di grab indonesia,neutral
394,Paling suka banget makan siang di sini ayam sa...,positive
592,Pelayanan bus DAMRI sangat baik,positive


In [13]:
test.head()

,text,label
id,,
411,"Dekat dengan hotel saya menginap, hanya ditemp...",positive
729,"Iya benar, dia sedang jaga warung.",neutral
373,Kangkungnya lumayan tapi kepiting saus padangn...,negative
262,Bertempat di braga city walk yang satu gedung ...,positive
177,Gianyar terima bantuan sosial 2018 sebesar rp ...,neutral


In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 219 to 719
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    500 non-null    object
 1   label   500 non-null    object
dtypes: object(2)
memory usage: 11.7+ KB


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 411 to 768
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    400 non-null    object
 1   label   400 non-null    object
dtypes: object(2)
memory usage: 9.4+ KB


## Preprocessing

In [16]:
preprocessor = TextPreprocessor(vocab_size=10000, max_length=100)
processed_data = preprocessor.preprocess_dataset(train, valid, test, use_vectorizer=False)

train_seq = processed_data['train_sequences']
val_seq = processed_data['val_sequences']
test_seq = processed_data['test_sequences']
train_labels = processed_data['train_labels']
val_labels = processed_data['val_labels']
test_labels = processed_data['test_labels']

vocab_size = processed_data['vocab_size']
num_classes = processed_data['num_classes']

print(f"Vocab size: {vocab_size}")
print(f"Num classes: {num_classes}")
print(f"Sequence shape: {train_seq.shape}")
print(f"Labels distribution: {np.bincount(train_labels)}")

Vocab size: 2796
Num classes: 3
Sequence shape: (500, 100)
Labels distribution: [192 119 189]


## Modelling

In [17]:
configs = {
    # Pengaruh jumlah layer (3 variasi)
    'rnn_1layer': {'hidden_sizes': [64], 'bidirectional': False},
    'rnn_2layer': {'hidden_sizes': [64, 32], 'bidirectional': False},
    'rnn_3layer': {'hidden_sizes': [64, 32, 16], 'bidirectional': False},
    
    # Pengaruh banyak cell (3 variasi)
    'rnn_cells_32': {'hidden_sizes': [32, 32], 'bidirectional': False},
    'rnn_cells_64': {'hidden_sizes': [64, 64], 'bidirectional': False},
    'rnn_cells_128': {'hidden_sizes': [128, 128], 'bidirectional': False},
    
    # Pengaruh arah (2 variasi)
    'rnn_unidirectional': {'hidden_sizes': [64, 32], 'bidirectional': False},
    'rnn_bidirectional': {'hidden_sizes': [64, 32], 'bidirectional': True}
}

In [18]:
training_results = {}

for name, config in configs.items():
    print(f"\nTraining {name}...")
    
    # Create and train
    model = create_keras_rnn(vocab_size, num_classes, config)
    
    history = model.fit(
        train_seq, train_labels,
        validation_data=(val_seq, val_labels),
        epochs=10, batch_size=32, verbose=1
    )
    
    # Evaluate
    y_pred = np.argmax(model.predict(test_seq), axis=1)
    macro_f1 = f1_score(test_labels, y_pred, average='macro')
    
    # Save
    model.save(f'../../models/{name}.h5')
    
    training_results[name] = {
        'config': config,
        'macro_f1': macro_f1,
        'history': history.history
    }
    
    print(f"✅ {name} - F1: {macro_f1:.4f}")


Training rnn_1layer...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.3310 - loss: 1.1105 - val_accuracy: 0.3800 - val_loss: 1.0813
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4802 - loss: 1.0120 - val_accuracy: 0.3700 - val_loss: 1.1509
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7061 - loss: 0.7850 - val_accuracy: 0.4100 - val_loss: 1.1573
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9083 - loss: 0.4690 - val_accuracy: 0.3900 - val_loss: 1.2227
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9935 - loss: 0.2204 - val_accuracy: 0.4600 - val_loss: 1.2008
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9975 - loss: 0.1041 - val_accuracy: 0.4600 - val_loss: 1.2994
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9990 - loss: 0.0581 - val_accuracy: 0.4500 - val_loss: 1.3197
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0384 - 

✅ rnn_1layer - F1: 0.4329

Training rnn_2layer...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.3641 - loss: 1.1673 - val_accuracy: 0.3800 - val_loss: 1.0806
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3670 - loss: 1.1388 - val_accuracy: 0.4000 - val_loss: 1.1410
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4944 - loss: 1.0256 - val_accuracy: 0.4600 - val_loss: 1.1511
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5558 - loss: 0.9478 - val_accuracy: 0.4300 - val_loss: 1.2856
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6701 - loss: 0.7342 - val_accuracy: 0.5100 - val_loss: 1.3623
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7057 - loss: 0.6517 - val_accuracy: 0.4300 - val_loss: 1.5872
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7667 - loss: 0.5617 - val_accuracy: 0.4400 - val_loss: 1.6479
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy

✅ rnn_2layer - F1: 0.3913

Training rnn_3layer...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.3176 - loss: 1.2354 - val_accuracy: 0.4000 - val_loss: 1.1028
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4668 - loss: 1.0632 - val_accuracy: 0.4100 - val_loss: 1.1835
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4821 - loss: 1.0167 - val_accuracy: 0.3900 - val_loss: 1.2388
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5981 - loss: 0.8772 - val_accuracy: 0.4900 - val_loss: 1.1934
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6630 - loss: 0.7592 - val_accuracy: 0.4500 - val_loss: 1.3126
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7934 - loss: 0.5739 - val_accuracy: 0.4100 - val_loss: 1.3591
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8674 - loss: 0.4594 - val_accuracy: 0.4500 - val_loss: 1.4708
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy

✅ rnn_3layer - F1: 0.4168

Training rnn_cells_32...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.3305 - loss: 1.1611 - val_accuracy: 0.3800 - val_loss: 1.0946
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4392 - loss: 1.0568 - val_accuracy: 0.3800 - val_loss: 1.1674
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7117 - loss: 0.7632 - val_accuracy: 0.3500 - val_loss: 1.2757
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9031 - loss: 0.4555 - val_accuracy: 0.3800 - val_loss: 1.3483
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9661 - loss: 0.1919 - val_accuracy: 0.4400 - val_loss: 1.3601
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.0849 - val_accuracy: 0.4400 - val_loss: 1.4527
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.0433 - val_accuracy: 0.4100 - val_loss: 1.6044
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accura

✅ rnn_cells_32 - F1: 0.3849

Training rnn_cells_64...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.3156 - loss: 1.1898 - val_accuracy: 0.4100 - val_loss: 1.1105
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4099 - loss: 1.0570 - val_accuracy: 0.4800 - val_loss: 1.0311
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7247 - loss: 0.7651 - val_accuracy: 0.3000 - val_loss: 1.4031
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8755 - loss: 0.4083 - val_accuracy: 0.3800 - val_loss: 1.6066
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9730 - loss: 0.1389 - val_accuracy: 0.3500 - val_loss: 1.9349
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9949 - loss: 0.0506 - val_accuracy: 0.3500 - val_loss: 2.3325
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 0.0268 - val_accuracy: 0.3100 - val_loss: 2.3861
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accu

✅ rnn_cells_64 - F1: 0.3523

Training rnn_cells_128...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.3671 - loss: 1.1491 - val_accuracy: 0.4400 - val_loss: 1.1100
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3114 - loss: 1.2129 - val_accuracy: 0.3900 - val_loss: 1.0749
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3781 - loss: 1.1565 - val_accuracy: 0.3300 - val_loss: 1.1515
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3861 - loss: 1.1421 - val_accuracy: 0.3200 - val_loss: 1.1124
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3538 - loss: 1.1970 - val_accuracy: 0.3800 - val_loss: 1.0993
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3405 - loss: 1.2161 - val_accuracy: 0.4100 - val_loss: 1.0756
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.3761 - loss: 1.1566 - val_accuracy: 0.4400 - val_loss: 1.0624
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - acc

✅ rnn_cells_128 - F1: 0.2346

Training rnn_unidirectional...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.3450 - loss: 1.1607 - val_accuracy: 0.3900 - val_loss: 1.0892
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4024 - loss: 1.0918 - val_accuracy: 0.4000 - val_loss: 1.1493
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6301 - loss: 0.8146 - val_accuracy: 0.2800 - val_loss: 1.4062
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7760 - loss: 0.5808 - val_accuracy: 0.4500 - val_loss: 1.4552
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9368 - loss: 0.2738 - val_accuracy: 0.4200 - val_loss: 1.6437
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9881 - loss: 0.1485 - val_accuracy: 0.4100 - val_loss: 1.8211
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9988 - loss: 0.0632 - val_accuracy: 0.4100 - val_loss: 1.9693
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

✅ rnn_unidirectional - F1: 0.3922

Training rnn_bidirectional...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - accuracy: 0.3467 - loss: 1.1513 - val_accuracy: 0.4300 - val_loss: 1.0573
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6127 - loss: 0.8414 - val_accuracy: 0.5400 - val_loss: 0.9613
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.8938 - loss: 0.4975 - val_accuracy: 0.5600 - val_loss: 0.9467
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9824 - loss: 0.1941 - val_accuracy: 0.5900 - val_loss: 0.9734
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9979 - loss: 0.0702 - val_accuracy: 0.5800 - val_loss: 1.0903
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 1.0000 - loss: 0.0290 - val_accuracy: 0.5700 - val_loss: 1.1304
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 1.0000 - loss: 0.0188 - val_accuracy: 0.5900 - val_loss: 1.1353
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/

✅ rnn_bidirectional - F1: 0.5213


## Compare models

In [19]:
def compare_implementations(model_path, config, test_data):
    """Compare Keras vs from scratch"""
    test_X, test_y = test_data
    
    # Load Keras model
    keras_model = tf.keras.models.load_model(model_path)
    
    # Create scratch model
    scratch_model = SimpleRNNModel(
        vocab_size=vocab_size,
        embedding_dim=128,
        hidden_sizes=config['hidden_sizes'],
        num_classes=num_classes,
        bidirectional=config['bidirectional']
    )
    
    # Load weights to scratch model
    scratch_model.load_keras_weights(model_path)
    
    # Get predictions
    keras_proba = keras_model.predict(test_X, verbose=0)
    scratch_proba = scratch_model.predict_proba(test_X)
    
    keras_pred = np.argmax(keras_proba, axis=1)
    scratch_pred = scratch_model.predict(test_X)
    
    # Calculate metrics
    keras_f1 = f1_score(test_y, keras_pred, average='macro')
    scratch_f1 = f1_score(test_y, scratch_pred, average='macro')
    
    max_diff = np.max(np.abs(keras_proba - scratch_proba))
    agreement = np.mean(keras_pred == scratch_pred)
    
    
    return {
        'keras_f1': keras_f1,
        'scratch_f1': scratch_f1,
        'max_diff': max_diff,
        'agreement': agreement
    }

In [20]:
test_data = (test_seq, test_labels)
comparison_results = {}

for name, config in configs.items():
    print(f"\nComparing {name}...")
    model_path = f'../../models/{name}.h5'
    
    result = compare_implementations(model_path, config, test_data)
    comparison_results[name] = result
    
    print(f"  Keras F1: {result['keras_f1']:.4f}")
    print(f"  Scratch F1: {result['scratch_f1']:.4f}")
    print(f"  Max diff: {result['max_diff']:.6f}")
    print(f"  Agreement: {result['agreement']:.4f}")


Comparing rnn_1layer...
Weights loaded from ../../models/rnn_1layer.h5


  Keras F1: 0.4329
  Scratch F1: 0.4329
  Max diff: 0.000001
  Agreement: 1.0000

Comparing rnn_2layer...
Weights loaded from ../../models/rnn_2layer.h5


  Keras F1: 0.3913
  Scratch F1: 0.3913
  Max diff: 0.000003
  Agreement: 1.0000

Comparing rnn_3layer...
Weights loaded from ../../models/rnn_3layer.h5


  Keras F1: 0.4168
  Scratch F1: 0.4168
  Max diff: 0.000013
  Agreement: 1.0000

Comparing rnn_cells_32...
Weights loaded from ../../models/rnn_cells_32.h5


  Keras F1: 0.3849
  Scratch F1: 0.3849
  Max diff: 0.000002
  Agreement: 1.0000

Comparing rnn_cells_64...
Weights loaded from ../../models/rnn_cells_64.h5


  Keras F1: 0.3523
  Scratch F1: 0.3523
  Max diff: 0.000004
  Agreement: 1.0000

Comparing rnn_cells_128...
Weights loaded from ../../models/rnn_cells_128.h5


  Keras F1: 0.2346
  Scratch F1: 0.2346
  Max diff: 0.000000
  Agreement: 1.0000

Comparing rnn_unidirectional...
Weights loaded from ../../models/rnn_unidirectional.h5


  Keras F1: 0.3922
  Scratch F1: 0.3922
  Max diff: 0.000002
  Agreement: 1.0000

Comparing rnn_bidirectional...
Weights loaded from ../../models/rnn_bidirectional.h5
  Keras F1: 0.5213
  Scratch F1: 0.5213
  Max diff: 0.000004
  Agreement: 1.0000
